In [1]:
from collections import defaultdict
from functools import lru_cache

from requests import Session

import pandas as pd

In [2]:
AUTH_KEY = "XeUIxlvO4CPc44NlLE3ncevDg7bAhp6CRy6zC9M2aQb2zGfys0M30eKwavFJSEJr"

read_prefix = "https://www.thebluealliance.com/api/v3/"

session = Session()
session.headers.update({"X-TBA-Auth-Key": AUTH_KEY, "X-TBA-Auth-Id": ""})

def get_tba(url):
    response = session.get(read_prefix + url)
    return response.json()

@lru_cache()
def get_district_points(key):
    data = get_tba(f"event/{key}/district_points")
    return {int(k[3:]): v["total"] for k, v in data["points"].items()}

In [3]:
responses = defaultdict(list)

for _, row in pd.read_csv("./contest_responses.csv").iterrows():
    submission = {
        "user": [row["Name"], row["Chief Delphi Username"], row["Team Affiliation"]],
        "ranks": [row["Rank " + str(i)] for i in range(1, 17)]
    }
    
    if submission["user"][0] == "EPA" and row["Event"] == "MAR Champs":
        submission["ranks"][0] = 2539
    
    responses[row["Event"]].append(submission)

In [4]:
event_name_to_key = {
    "PNW Champs": "2023pncmp",
    "Michigan Champs - APTIV Division": "2023micmp3",
    "Indiana Champs": "2023incmp",
    "Michigan Champs - DTE Division": "2023micmp1",
    "Texas Champs - APOLLO Division": "2023txcmp1",
    "New England Champs - WILSON Division": "2023necmp2",
    "New England Champs - MEIR Division": "2023necmp1",
    "MAR Champs": "2023mrcmp",
    "Ontario Champs - TECHNOLOGY Division": "2023oncmp1",
    "CHS Champs": "2023chcmp",
    "Michigan Champs - CONSUMER Division": "2023micmp4",
    "Ontario Champs - SCIENCE Division": "2023oncmp2",
    "Texas Champs - MERCURY Division": "2023txcmp2",
    "Michigan Champs - FORD Division": "2023micmp2",
    "PCH Champs": "2023gacmp",
}

for comp in responses:
    print(comp)
    team_points = get_district_points(event_name_to_key[comp])
    for response in responses[comp]:
        score = [(16 - i) * team_points.get(x, 0) for i, x in enumerate(response["ranks"])]
        response["score"] = score
        
    for response in sorted(responses[comp], key=lambda x: -sum(x["score"])):
        print(response["user"])
        print(" ".join([str(x).ljust(4, " ") for x in response["ranks"]]))
        print(" ".join([str(x).ljust(4, " ") for x in response["score"]]), sum(response["score"]))
        print()
        
    print()
    print()
    print()

PNW Champs
['Chris', 'Radicalthief', '955']
2910 2046 2930 2147 2521 7461 3663 955  3636 1540 3218 2990 7627 488  360  2471
3504 2610 1428 858  1188 957  1080 1188 936  924  594  330  120  252  204  27   16200

['Kevin Emery', 'Kpchem', 'None']
2910 2930 2046 3663 7461 2412 1318 7627 2147 360  2521 3218 2990 2811 1425 3636
3504 1530 2436 1404 1044 1485 1200 270  528  714  594  495  264  126  240  117  15951

[nan, 'andej', '997']
2910 2046 2521 3663 2147 2930 1318 955  7627 1425 2990 1318 7461 2471 2811 3218
3504 2610 1386 1404 792  1122 1200 1188 240  840  396  600  348  81   84   99   15894

['seth', 'sethP', '8248']
2910 2046 3663 2521 7461 3218 2147 2990 2930 1318 7627 3636 488  1778 2811 360 
3504 2610 1512 1287 1044 1089 660  594  816  840  180  585  336  279  84   102  15522

['David Sergent', 'MadDad2930', nan]
2910 2930 2046 3663 2147 3218 2521 7627 2522 1318 1425 7461 1778 3636 360  4911
3504 1530 2436 1404 792  1089 990  270  336  840  720  435  372  351  204  27   15300

['

['James', 'MARS_James', '179']
2539 5895 341  1923 1391 316  3314 1807 1403 1676 25   11   2495 1218 4373 834 
2640 2835 1722 2106 1512 1287 2010 918  936  924  648  555  120  288  162  99   18762

['Erik Yngstrom', 'Boganafoganasee', '1768']
5895 1923 2539 316  25   1676 3314 1403 11   1218 1391 4373 1807 103  1640 2590
3024 2430 2310 1521 1296 1452 2010 1053 888  672  756  405  408  342  102  93   18762

['EPA', nan, nan]
2539 5895 341  1391 316  1923 1807 1676 3314 25   4373 834  1403 1218 11   103 
2640 2835 1722 1638 1404 1782 1020 1188 1608 756  486  495  468  288  222  114  18666

['Jeff Grabner ', 'Juggernaut', '1391']
341  2539 1391 5895 1923 3314 1807 316  1676 4373 25   1218 103  11   1640 1403
1968 2475 1764 2457 1944 2211 1020 1053 1056 567  648  480  456  333  102  117  18651

['Cole', 'Cole', '4342']
2539 1923 341  5895 25   1676 1391 1807 316  103  3314 1403 4373 11   4342 2590
2640 2430 1722 2457 1296 1452 1260 918  936  798  1206 585  324  333  198  93   18648

['John

In [5]:
for response in sorted(responses["Indiana Champs"], key=lambda x: -sum(x["score"])):
    name = response["user"][0] if type(response["user"][1]) == float else response["user"][1]
    print("| " + "FIN" + " | " + str(name) + " | " + str(sum(response["score"])))              

| FIN | nan | 17637
| FIN | Ehochstein | 16581
| FIN | BlueEpiphone | 16272
| FIN | AdamB | 16134
| FIN | becca461 | 15954
| FIN | Taylor | 15846
| FIN | EPA | 15792
| FIN | INsight | 15318
| FIN | Charvey829 | 14622
